# Merged Flame Graph

How to generate a Flame Graph out of a Chrome profile. This Flame Graph includes both Trace Event traces and Inspector 1.2 profiles.. Example file was recorded in Chrome.

In [1]:
f = open('examples/Profile-20180716T115056', 'r')

In [2]:
import json

profile = json.load(f)

In [3]:
start = None
end = None

js_invocation_events = (
    'RunMicrotasks',
    'FunctionCall',
    'EvaluateScript',
    'v8.evaluateModule',
    'EventDispatch',
    'V8.Execute'
)

profile_events = []
trace_events = []
js_stack_events = []
is_js_stack = False

for row in profile:
    if row['ph'] != 'M':
        if start is None or int(row['ts']) < start:
            start = int(row['ts'])
        if end is None or int(row['ts']) > end:
            end = int(row['ts'])
    cat = row['cat'].split(',')
    if 'devtools.timeline' in cat:
        if row['ph'] in ['B', 'E', 'X']:
            if row['name'] in js_invocation_events:
                if row['ph'] == 'B':
                    is_js_stack = True
                    trace_events.append(row)
                elif row['ph'] == 'E':
                    is_js_stack = False
                    trace_events.append(row)
            else:
                if is_js_stack:
                    js_stack_events.append(row)
                else:
                    trace_events.append(row)
    if row['ph'] == 'I' and row['name'] == 'CpuProfile':
        profile_events.append(row)

In [4]:
def parse_nodes(data):
    nodes = {}
    for node in data['cpuProfile']['nodes']:
        node_id = node['id']
        function_name = node['callFrame']['functionName']
        url = node['callFrame']['url']
        line_number = node['callFrame']['lineNumber']
        children = node.get('children')
        hit_count = node.get('hitCount')
        nodes[node_id] = {'function_name': function_name, 'url': url, 'line_number': line_number, 'hit_count': hit_count, 'children': children}
    return nodes

In [5]:
import copy

def generate_stacks(node_id, nodes, stacks, current_stack):
    node = nodes[node_id] # break in case id doesn't exist
    if node['function_name'] == '':
        node['function_name'] = '(anonymous)'
    if node['function_name'] != '(root)':
        current_stack.append(node['function_name'])
        stacks[node_id] = current_stack
    if node['children']:
        for child in node['children']:
            generate_stacks(child, nodes, stacks, copy.copy(current_stack))
    del nodes[node_id]

In [6]:
def get_events(pid, tid, samples, time_deltas, start_time, stacks, ignore_ids):
    events = []
    current_time = start_time + time_deltas[0]
    for index, sample in enumerate(samples):
        if index == (len(samples) - 1): # last sample
            break
        delta = time_deltas[index + 1]
        if delta < 0: # TODO: find a better way to deal with negative time deltas
            delta = 0
            continue
        current_time += delta
        if sample not in ignore_ids:
            stack = stacks[sample]
            events.append({
                'pid': pid,
                'tid': tid,
                'name': stack[0],
                'cat': 'JSFrame',
                'ph': 'X',
                'ts': current_time,
                'dur': delta,
                'stack': stack
            })
    return events

In [7]:
def get_meta_ids(nodes):
    program_node_id = None
    idle_node_id = None
    gc_node_id = None
    for key, node in nodes.items():
        if node['function_name'] == '(program)':
            program_node_id = key
        elif node['function_name'] == '(idle)':
            idle_node_id = key
        elif node['function_name'] == '(garbage collector)':
            gc_node_id = key
    return program_node_id, idle_node_id, gc_node_id

In [8]:
cpuprofile_events = []

for profile in profile_events:
    pid = profile['pid']
    tid = profile['tid']
    data = profile['args']['data']
    root_id = data['cpuProfile']['nodes'][0]['id']
    nodes = parse_nodes(data)
    ignore_ids = get_meta_ids(nodes)
    stacks = {}
    generate_stacks(root_id, nodes, stacks, [])
    cpuprofile_events += get_events(pid, tid, data['cpuProfile']['samples'], data['cpuProfile']['timeDeltas'], data['cpuProfile']['startTime'], stacks, ignore_ids)


In [9]:
# TODO: subtract deltas to find error rate

def add_to_stack(start_time, end_time, name, cpuprofile_events_index):
    dur = end_time - start_time
    start_event_index = None
    end_event_index = None
    for i in range(cpuprofile_events_index, len(cpuprofile_events)):
        if cpuprofile_events[i]['ts'] > start_time:
            if start_event_index is None:
                start_event_index = i
        if cpuprofile_events[i]['ts'] > end_time:
            end_event_index = i
            break
        cpuprofile_events_index = i
    if start_event_index is not None and end_event_index is not None:
        if (end_event_index - start_event_index) > 1:
            for i in range(start_event_index, end_event_index):
                cpuprofile_events[i]['stack'].append(name)
        elif (end_event_index - start_event_index) == 0:
            if (dur / cpuprofile_events[start_event_index]['dur']) > 0.5:
                # ignore if event duration is less than 50% of the sampled delta
                cpuprofile_events[start_event_index]['stack'].append(name)
        else:
            raise Exception("event index is reversed", start_event_index, end_event_index) 
    else:
        # raise Exception("could not find start or end index", start_time, end_time)
        print("could not find start or end index", start_time, end_time)
    return cpuprofile_events_index

In [10]:
# TODO: handle multi-level stacks

is_stack_open = False
ts = None
cat = None
name = None

cpuprofile_events_index = 0

for event in js_stack_events:
    if event['ph'] == 'B':
        if is_stack_open:
            raise Exception("multi-level stack found")
        is_stack_open = True
        ts = event['ts']
        cat = event['cat']
        name = event['name']
    elif event['ph'] == 'E':
        if not is_stack_open:
            raise Exception("end event found without an open stack")
        if event['cat'] != cat or event['name'] != name:
            raise Exception("end event with different cat or name")
        cpuprofile_events_index = add_to_stack(ts, event['ts'], event['name'], cpuprofile_events_index)
        is_stack_open = False
        ts = None
        cat = None
        name = None
    elif event['ph'] == 'X':
        start_time = event['ts']
        end_time = event['ts'] + event['dur']
        cpuprofile_events_index = add_to_stack(start_time, end_time, event['name'], cpuprofile_events_index)
         

could not find start or end index 24636966935 24636967192
could not find start or end index 24636969766 24636972042
could not find start or end index 24636972121 24636972372
could not find start or end index 24636972430 24636973866
could not find start or end index 24636974867 24636975135


In [11]:
# time skew

print((cpuprofile_events[-1]['ts'] - js_stack_events[-1]['ts']) / 1000000, "seconds")

0.269614 seconds


In [12]:
# TODO: add cpuprofile_events to trace_events

In [13]:
root = {'name': 'root', 'value': 0, 'children': []}
open_partial_slices = {}

# TODO: handle CPU time differences, where "E" comes before "B"

def get_child_slice(parent_slice, name):
    for index, child in enumerate(parent_slice['children']):
        if child['name'] == name:
            return parent_slice['children'].pop(index)
    return None

def insert_slice(parent_slice, new_slice):
    child_slice = get_child_slice(parent_slice, new_slice['name'])
    if child_slice is None:
        child_slice = {'name': new_slice['name'], 'value': 0, 'children': []}
    for child in new_slice['children']:
        insert_slice(child_slice, child)
    child_slice['value'] += new_slice['value']
    parent_slice['children'].append(child_slice)

def check_thread(pid, tid):
    if pid not in open_partial_slices:
        open_partial_slices[pid] = {}
    if tid not in open_partial_slices[pid]:
        open_partial_slices[pid][tid] = []

def begin_slice(pid, tid, cat, name, ts, tts):
    check_thread(pid, tid)
    open_partial_slices[pid][tid].append({'pid': pid, 'tid': tid, 'cat': cat, 'name': name, 'ts': ts, 'tts': tts, 'children': []})

def end_slice(pid, tid, cat, name, ts, tts):
    partial_slice_count = len(open_partial_slices[pid][tid])
    if partial_slice_count > 0:
        current_slice = open_partial_slices[pid][tid].pop()
        if current_slice['cat'] != cat or current_slice['name'] != name:
            raise Exception("ending slice with different cat or name", pid, tid, cat, name, ts)
        current_slice['dur'] = ts - current_slice['ts']
        if tts is not None and current_slice['tts'] is not None:
            current_slice['tdur'] = tts - current_slice['tts']
        if 'tdur' in current_slice:
            current_slice['value'] = current_slice['tdur']
        else:
            current_slice['value'] = current_slice['dur']
        partial_slice_count = len(open_partial_slices[pid][tid])
        if partial_slice_count > 0:
            open_partial_slices[pid][tid][partial_slice_count - 1]['children'].append(current_slice)
        else:
            insert_slice(root, current_slice)
    else:
        raise Exception("end_slice called without an open slice", pid, tid, cat, name, ts)

In [14]:
# TODO: handle "sf" and "stack" properties on Duration Events

for row in iter(cpuprofile_events):
    if row['ph'] == 'B' or row['ph'] == 'E':
        if row['ph'] == 'B':
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
        elif row['ph'] == 'E':
            end_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
    elif row['ph'] == 'X':
        end_tts = None
        if 'tts' in row and 'tdur' in row:
            end_tts = row['tts'] + row['tdur']
        if 'dur' in row and row['dur'] > 0:
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
            end_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'] + row['dur'], end_tts)

In [15]:
import json

with open('merged.json', 'w') as file:
     file.write(json.dumps(root))